Ici on importe les librairies nécessaires
<ul>
    <li>`import tensorflow as tf` -> bibliothèque open-source de calcul numérique </li>
    <li>`from tensorflow.keras import layers,models` -> interface haut niveau pour construire des réseaux de neurones</li>
    <li>`from sklearn.model_selection import train_test_split` -> fonction qui permet de séparer les données en un ensemble d\'entraînement et un ensemble de validation </li>
</ul>

In [3]:

import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split


utilisation de la fonction `image_dataset_from_directory` de TensorFlow pour charger les données d'entraînement à partir d'un répertoire contenant des images
- `train_data`: le chemin vers le répertoire contenant les données d'entraînement.
- `batch_size`: taille des batchs sert à entraîner le modèle. ici un batch = 32 images.
- `image_size`: la taille de chaque image dans les données d'entraînement (224 x 224 pixels).
- `validation_split`: le ratio de données à utiliser pour la validation. 20% des données d'entraînement seront utilisées pour la validation.
- `seed`: la seed utilisée pour le shuffle. permet que les données sont mélangées de la même manière à chaque fois que le code est exécuté.
- `subset`: le sous-ensemble de données à utiliser.
- `shuffle`: si les données doivent être mélangées à chaque époque.Ici données doivent être mélangées avant chaque époque.

In [4]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    '../train',  
    batch_size=32,          
    image_size=(224, 224),  
    validation_split=0.2,   
    seed=42,                
    subset='training',      
    shuffle=True            
)

Found 5216 files belonging to 2 classes.
Using 4173 files for training.


Les données de validation sont utilisées pour évaluer la performance du modèle pendant l'entraînement. 
Le modèle est entraîné sur les données d'entraînement, puis il est évalué sur les données de validation à chaque époque. 
Cela permet de détecter 
    - si le modèle est en train de s'adapter trop aux données d'entraînement et ne généralise pas bien à de nouvelles données
    - s'il ne capture pas toutes les informations pertinentes des données d'entraînement.
    Les données de validation permettent donc de surveiller la performance du modèle pendant l'entraînement et de l'améliorer en conséquence.

In [5]:

val_data = tf.keras.preprocessing.image_dataset_from_directory(
    '../val',
    batch_size=32,
    image_size=(224, 224),
    validation_split=0.2,
    seed=42, 
    subset='validation',
    shuffle=True
)


Found 16 files belonging to 2 classes.
Using 3 files for validation.


`test_data` est utilisé pour évaluer les performances du modèle sur des données inédites,
 c'est-à-dire des données que le modèle n'a pas vues lors de l'entraînement et de la validation. 
 il est donc utiilisé pour tester sa capacité à produire des prédictions précises sur des données qu'il n'a jamais vues auparavant.

In [6]:
test_data = tf.keras.preprocessing.image_dataset_from_directory(
    '../test',
    batch_size=32,
    image_size=(224, 224)
)


Found 624 files belonging to 2 classes.


La méthode `prefetch()` est utilisée pour améliorer les performances du modèle en préchargeant un certain nombre de données en mémoire avant leur utilisation par le modèle.

ici le modèle préchargera 32 images à la fois en mémoire avant leur utilisation par le modèle.

In [7]:
train_data = train_data.prefetch(buffer_size=32)
val_data = val_data.prefetch(buffer_size=32)
test_data = test_data.prefetch(buffer_size=32)


le modèle de réseau de neurones convolutionnels (CNN) sert à entraîner le modèle. 
Voici une explication ligne par ligne :

- `models.Sequential()` crée un modèle séquentiel qui permet de définir les couches du réseau dans l'ordre.
- `layers.experimental.preprocessing.Rescaling(1./255)` normalise les valeurs de pixel de l'image pour qu'elles soient comprises entre 0 et 1.
- `layers.Conv2D(16, (3,3), activation='relu', input_shape=(224,224,3))` définit une couche de convolution avec 16 filtres de taille (3,3) et une fonction d'activation relu. 
- `input_shape=(224,224,3)` indique que l'entrée de la couche est une image RGB de taille 224x224.
- `layers.MaxPooling2D((2,2))` sert à réduire la taille de l'image.

En résumé, ce modèle CNN est conçu pour extraire des caractéristiques d'images et les utiliser pour classer des images en deux catégories. 
 - La première couche normalise les pixels de l'image,
 -  suivie de trois couches de convolution et de max-pooling pour extraire des caractéristiques d'image, 
 - puis une couche dense pour classifier l'image.

In [8]:
model = models.Sequential([
    layers.experimental.preprocessing.Rescaling(1./255),
    layers.Conv2D(16, (3,3), activation='relu', input_shape=(224,224,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])


sert à compiler le modèle et définir les paramètres de l'apprentissage. 

- `optimizer` : il s'agit de l'algorithme d'optimisation utilisé pour mettre à jour les poids du modèle lors de l'apprentissage.
- `loss` : c'est la fonction de perte utilisée pour calculer l'erreur du modèle lors de l'apprentissage. 
- `metrics` : il s'agit de la métrique utilisée pour évaluer les performances du modèle.    

In [9]:

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


 sert à entraîner le modèle sur les données d'entraînement (train_data) et la validation sera effectuée sur les données de validation (val_data). 
 epochs=10 indique qu'on entraîne le modèle pendant 10 époques

In [10]:
history = model.fit(train_data, validation_data=val_data, epochs=10)


Epoch 1/10
131/131 [==============================] - 230s 2s/step - loss: 0.2495 - accuracy: 0.8924 - val_loss: 0.2296 - val_accuracy: 1.0000
Epoch 2/10
131/131 [==============================] - 198s 1s/step - loss: 0.1118 - accuracy: 0.9573 - val_loss: 0.1074 - val_accuracy: 1.0000
Epoch 3/10
131/131 [==============================] - 193s 1s/step - loss: 0.0824 - accuracy: 0.9708 - val_loss: 0.0942 - val_accuracy: 1.0000
Epoch 4/10
131/131 [==============================] - 195s 1s/step - loss: 0.0597 - accuracy: 0.9796 - val_loss: 0.3287 - val_accuracy: 0.6667
Epoch 5/10
131/131 [==============================] - 184s 1s/step - loss: 0.0488 - accuracy: 0.9837 - val_loss: 0.2470 - val_accuracy: 0.6667
Epoch 6/10
131/131 [==============================] - 161s 1s/step - loss: 0.0460 - accuracy: 0.9839 - val_loss: 0.0489 - val_accuracy: 1.0000
Epoch 7/10
131/131 [==============================] - 105s 788ms/step - loss: 0.0382 - accuracy: 0.9859 - val_loss: 0.0477 - val_accuracy: 1.0

fonction qui retourne les valeurs de la fonction de perte et de la métrique choisie sur les données de test.

In [11]:
test_loss, test_acc = model.evaluate(test_data)

20/20 [==============================] - 11s 413ms/step - loss: 1.7205 - accuracy: 0.7516


on affiche la valeur de l'accuracy sur les données de test (test_acc) pour évaluer la performance finale du modèle.

In [12]:

print('Test accuracy:', test_acc)

Test accuracy: 0.7516025900840759
